In [1]:
# """
# Main file for the BarModels directory
# """
#---------------------------- Imports basic -------------------------------
import sys
import os
import logging
import time

import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')
startTime = time.time()
root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))
              # root = '/home/labs/mayalab/barc/MSc_studies/ML_Project'
if root not in sys.path:
    sys.path.append(root)
else:
    pass

if __name__ == "__main__":
    here = '/home/labs/mayalab/barc/MSc_studies/ML_Project/pyScripts/BarModels'
else:
    here = os.path.dirname(os.path.abspath(__name__))

%reload_ext autoreload
%autoreload 2
#---------------------------- Imports for the model -------------------------------
from pyScripts.BarModels.Rendom_forest_BC import Rendom_forest_classification_BC_defultParams
from pyScripts.BarModels.Rendom_forest_BC import Rendom_forest_classification_BC_useingGridSearchCV
from pyScripts.BarModels.Rendom_forest_BC import Rendom_forest_classification_BC_useing_Optuna

#---------------------------- Logger -------------------------------
def initialize_logging():
    log_directory = os.path.join(here, "logs")
    if not os.path.exists(log_directory):
        os.makedirs(log_directory)
    logging.basicConfig(filename=os.path.join(log_directory, f"{time.strftime('%Y-%m-%d_%H-%M-%S')}.log"),
                         level=logging.INFO,
                         format="%(asctime)s - %(levelname)s - %(message)s")

def log_message(message):
    current_time = time.localtime() 
    time_string = time.strftime("%Y-%m-%d %H:%M:%S", current_time)
    logging.info(f"{time_string} - {message}")
    logging.getLogger().handlers[0].flush()

# Initialize logging
initialize_logging()
# Log a message
log_message(f" the logger is working, the time is: {time.time()-startTime}")
log_message(f"starting to load the data, the time is: {time.time()-startTime}")
# ---------------------------- data incoming -------------------------------
def load_data():
    # Load the data
    X_train = pd.read_csv(os.path.join(here, 'X_train_df.csv'))
    y_train = pd.read_csv(os.path.join(here, 'y_train.csv'))
    X_test = pd.read_csv(os.path.join(here, 'X_test_df.csv'))
    y_test = pd.read_csv(os.path.join(here, 'y_test.csv'))

    # Compare the columns of X_train and X_test
    train_cols = set(X_train.columns)
    test_cols = set(X_test.columns)
    extra_cols = train_cols - test_cols

    # Remove any identified extra columns from X_train
    if extra_cols:
        X_train = X_train.drop(columns=list(extra_cols))

    # Convert DataFrame to numpy arrays and flatten
    X_train_np = X_train.values  # Converts DataFrame to numpy array
    y_train_np = y_train.values.ravel()  # Converts DataFrame to numpy array and flattens it
    X_test_np = X_test.values  # Converts DataFrame to numpy array
    y_test_np = y_test.values.ravel()  # Converts DataFrame to numpy array and flattens it

    return X_train_np, y_train_np, X_test_np, y_test_np

X_train_np, y_train, X_test_np, y_test = load_data()
log_message(f"finished loading the data, the time is: {time.time()-startTime}")


ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [8]:
import tensorflow as tf


[autoreload of numpy.core.overrides failed: Traceback (most recent call last):
  File "/apps/easybd/easybuild/software/IPython/7.26.0-GCCcore-11.2.0/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/apps/easybd/easybuild/software/IPython/7.26.0-GCCcore-11.2.0/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/labs/mayalab/barc/.conda/envs/ml-gpu/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/labs/mayalab/barc/.conda/envs/ml-gpu/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/labs/mayalab/barc/.conda/envs/ml-gpu/l

In [17]:
X_train_np

ValueError: Only callable can be used as callback

In [13]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import fashion_mnist

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the pixel values of the images to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# Build the model
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Convert each 28x28 image into a 784-dimensional vector
    Dense(128, activation='relu'),  # A dense layer with 128 units and ReLU activation
    Dense(10, activation='softmax')  # A dense layer with 10 units and softmax activation for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

4422102/4422102 [==============================] - 1s 0us/step


2024-04-22 10:04:24.834401: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46526 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:88:00.0, compute capability: 8.6
2024-04-22 10:04:24.835307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46496 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:cd:00.0, compute capability: 8.6


Epoch 1/10


2024-04-22 10:04:44.576765: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1875/1875 [==============================] - 17s 1ms/step - loss: 0.5029 - accuracy: 0.8243
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3753 - accuracy: 0.8646
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3380 - accuracy: 0.8768
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3111 - accuracy: 0.8859
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2950 - accuracy: 0.8913
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2769 - accuracy: 0.8973
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2667 - accuracy: 0.9015
Epoch 8/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2562 - accuracy: 0.9037
Epoch 9/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2465 - accuracy: 0.9084
Epoch 10/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2363 - accura

NameError: name 'operator' is not defined